# XGBoost Model Implementation with CPU Optimization

This notebook implements an XGBoost-based model for soccer match draw prediction with CPU optimization. The implementation includes:

- Model creation and configuration
- Training with early stopping
- Threshold optimization
- Hyperparameter tuning
- Model evaluation
- MLflow integration for experiment tracking

## Setup and Imports

In [179]:
# Block 1: Imports and Setup
import numpy as np
import pandas as pd
from pathlib import Path
import xgboost as xgb
import joblib
import json
import os
import sys
import time
import optuna
import mlflow
import mlflow.xgboost
import random
from typing import Any, Dict, Tuple
from datetime import datetime
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner
from sklearn.metrics import roc_auc_score
import yaml

# Add project root to Python path
project_root = str(Path().absolute().parent.parent.parent.parent)
if project_root not in sys.path:
    sys.path.append(project_root)
os.environ["PYTHONPATH"] = project_root + os.pathsep + os.environ.get("PYTHONPATH", "")
os.environ["ARROW_S3_DISABLE"] = "1"

from utils.logger import ExperimentLogger
experiment_name = "xgboost_soccer_prediction"
logger = ExperimentLogger(experiment_name)

from utils.create_evaluation_set import setup_mlflow_tracking
from models.StackedEnsemble.utils.metrics import calculate_metrics
from models.StackedEnsemble.shared.data_loader import DataLoader

# Load data
mlruns_dir = setup_mlflow_tracking(experiment_name)
dataloader = DataLoader()
X_train, y_train, X_test, y_test, X_eval, y_eval = dataloader.load_data()


2025-02-20 21:34:39,783 | INFO     | xgboost_soccer_prediction | Setting up MLflow tracking for experiment: xgboost_soccer_prediction
mlflow local_path_uri: c:/Users/szita/Documents/TheDrawCode/mlruns
2025-02-20 21:34:39,904 | INFO     | xgboost_soccer_prediction | Using existing experiment: xgboost_soccer_prediction experiment_id: 600562561289637747
2025-02-20 21:34:39,904 | INFO     | xgboost_soccer_prediction | MLflow tracking configured successfully at: c:/Users/szita/Documents/TheDrawCode/mlruns
2025-02-20 21:34:39,904 | INFO     | xgboost_soccer_prediction | Loading data splits according to ensemble strategy
2025-02-20 21:34:39,904 | INFO     | xgboost_soccer_prediction | Returning features common to all models
2025-02-20 21:34:39,915 | INFO     | xgboost_soccer_prediction | Loaded 102 selected features
2025-02-20 21:34:39,980 | INFO     | xgboost_soccer_prediction | Loaded training data from parquet: c:\Users\szita\Documents\TheDrawCode\data\api_training_final.parquet
2025-02-20

In [180]:
# Block 2: Configuration Loading
def load_hyperparameter_space():
    """Define hyperparameter space for optimization."""
    try:
        hyperparameter_space = {
            'learning_rate': {
                'type': 'float',
                'low': 0.02,
                'high': 0.06,
                'log': True
            },
            'max_depth': {
                'type': 'int', 
                'low': 8,
                'high': 14
            },
            'min_child_weight': {
                'type': 'int',
                'low': 50,
                'high': 100
            },
            'subsample': {
                'type': 'float',
                'low': 0.4,
                'high': 0.7
            },
            'colsample_bytree': {
                'type': 'float',
                'low': 0.6,
                'high': 0.8
            },
            'reg_alpha': {
                'type': 'float',
                'low': 0.01,
                'high': 1.0,
                'log': True
            },
            'reg_lambda': {
                'type': 'float',
                'low': 7.0,
                'high': 12.0,
                'log': True
            },
            'gamma': {
                'type': 'float',
                'low': 0.1,
                'high': 1.0
            },
            'early_stopping_rounds': {
                'type': 'int',
                'low': 300,
                'high': 350
            },
            'scale_pos_weight': {
                'type': 'float',
                'low': 2.4,
                'high': 2.6
            },
            'rate_drop': {
                'type': 'float',
                'low': 0.03,
                'high': 0.3
            },
            'skip_drop': {
                'type': 'float',
                'low': 0.2,
                'high': 0.8
            }
        }
        
        # hyperparameter_space = {
        #     'learning_rate': {
        #         'type': 'float',
        #         'low': 0.04,       # Focus around ~0.046
        #         'high': 0.05,
        #         'log': True
        #     },
        #     'max_depth': {
        #         'type': 'int',
        #         'low': 10,         # Narrow around best value 11
        #         'high': 12
        #     },
        #     'min_child_weight': {
        #         'type': 'int',
        #         'low': 70,         # Focus near best 72
        #         'high': 75
        #     },
        #     'subsample': {
        #         'type': 'float',
        #         'low': 0.45,       # Focus around 0.4639
        #         'high': 0.48
        #     },
        #     'colsample_bytree': {
        #         'type': 'float',
        #         'low': 0.65,       # Focus near best 0.6671
        #         'high': 0.70
        #     },
        #     'reg_alpha': {
        #         'type': 'float',
        #         'low': 0.7,        # Around best 0.7545
        #         'high': 0.8,
        #         'log': True
        #     },
        #     'reg_lambda': {
        #         'type': 'float',
        #         'low': 9.0,        # Nearly best 9.32
        #         'high': 10.0,
        #         'log': True
        #     },
        #     'gamma': {
        #         'type': 'float',
        #         'low': 0.75,       # Around best 0.7820
        #         'high': 0.85
        #     },
        #     'early_stopping_rounds': {
        #         'type': 'int',
        #         'low': 300,        # Focused around best 327
        #         'high': 350
        #     },
        #     'scale_pos_weight': {
        #         'type': 'float',
        #         'low': 2.4,        # Around best 2.4845
        #         'high': 2.6
        #     }
        # }
        
        return hyperparameter_space
    except Exception as e:
        logger.error(f"Error creating hyperparameter space: {str(e)}")
        return None

hyperparameter_space = load_hyperparameter_space()


In [181]:
# Block 3: Model Creation
def create_model(**kwargs):
    """Create and configure XGBoost model instance."""
    try:
        params = {
            'objective': 'binary:logistic',
            'eval_metric': ['logloss', 'auc'],
            'tree_method': 'hist',
            'n_jobs': -1,
            'verbosity': 0
        }
        
        # Update with provided parameters
        params.update(kwargs)
        
        # Create model
        model = xgb.XGBClassifier(**params)
        
        return model
        
    except Exception as e:
        logger.error(f"Error creating XGBoost model: {str(e)}")
        raise


In [182]:
# Block 4: Data Conversion
def convert_to_model_format(X: pd.DataFrame, y: pd.Series = None):
    """Convert data to XGBoost DMatrix format."""
    if X is None:
        raise ValueError("The feature dataset X must not be None.")
    
    try:
        if y is not None:
            dmatrix = xgb.DMatrix(X, label=y)
        else:
            dmatrix = xgb.DMatrix(X)
        return dmatrix
    except Exception as e:
        logger.error(f"Error converting data to DMatrix: {str(e)}")
        raise


In [183]:
# Block 6: Prediction Functions
def predict(model, X, threshold=0.5):
    """Generate predictions using trained model."""
    if model is None:
        raise RuntimeError("Model must be trained before prediction")
        
    try:
        probas = model.predict_proba(X)[:, 1]
        return (probas >= threshold).astype(int)
        
    except Exception as e:
        logger.error(f"Error in model prediction: {str(e)}")
        return np.zeros(len(X))

def predict_proba(model, X):
    """Generate probability predictions."""
    if model is None:
        raise RuntimeError("Model must be trained before prediction")
        
    try:
        return model.predict_proba(X)[:, 1]
        
    except Exception as e:
        logger.error(f"Error in probability prediction: {str(e)}")
        return np.zeros(len(X))


In [184]:
def evaluate(model, X: Any, y: Any, best_threshold: float) -> Dict[str, float]:
    """Evaluate model performance on given data."""
    if model is None:
        raise RuntimeError("Model must be trained before evaluation")
    
    try:
        # Get probability predictions
        y_prob = model.predict_proba(X)[:, 1]
        
        # Get binary predictions using best threshold
        y_pred = (y_prob >= best_threshold).astype(int)
        
        # Calculate metrics
        tp = np.sum((y == 1) & (y_pred == 1))
        fp = np.sum((y == 0) & (y_pred == 1))
        fn = np.sum((y == 1) & (y_pred == 0))
        
        metrics = {
            'precision': tp / (tp + fp + 1e-10),
            'recall': tp / (tp + fn + 1e-10),
            'f1': 2 * tp / (2 * tp + fp + fn + 1e-10),
            'auc': roc_auc_score(y, y_prob),
            'brier_score': np.mean((y_prob - y) ** 2),
            'threshold': best_threshold
        }
        
        return metrics
        
    except Exception as e:
        logger.error(f"Error in model evaluation: {str(e)}")
        return {
            'precision': 0.0,
            'recall': 0.0,
            'f1': 0.0,
            'auc': 0.0,
            'brier_score': 1.0,
            'threshold': best_threshold
        }


In [185]:
def optimize_threshold(model, y_true: np.ndarray, y_prob: np.ndarray) -> float:
    """Optimize prediction threshold with focus on precision while maintaining recall above 15%."""
    try:
        best_threshold = 0.5
        best_precision = 0.0
        
        # Search through thresholds
        for threshold in np.linspace(0.3, 0.8, 51):
            y_pred = (y_prob >= threshold).astype(int)
            
            # Calculate confusion matrix components
            tp = np.sum((y_true == 1) & (y_pred == 1))
            fp = np.sum((y_true == 0) & (y_pred == 1))
            fn = np.sum((y_true == 1) & (y_pred == 0))
            
            precision = tp / (tp + fp + 1e-10)
            recall = tp / (tp + fn + 1e-10)
            
            # Only consider thresholds that maintain recall above 15%
            if recall >= 0.15:
                if precision > best_precision:
                    best_precision = precision
                    best_threshold = threshold
        
        logger.info(f"Optimized threshold: {best_threshold:.3f} with precision: {best_precision:.3f}")

        metrics = evaluate(model, X_eval, y_eval, best_threshold)
        return metrics
        
    except Exception as e:
        logger.error(f"Error optimizing threshold: {str(e)}")
        return 0.5


In [186]:
# Block 5: Training Function
def train_model(X_train, y_train, X_test, y_test, X_eval, y_eval, **kwargs):
    """Train XGBoost model with early stopping."""
    try:
        # Create model with remaining parameters
        model = create_model(**kwargs)
        
        # Create eval set for early stopping
        eval_set = [(X_test, y_test)]
        
        # Fit model with early stopping
        model.fit(
            X_train, y_train,
            eval_set=eval_set,
            verbose=False
        )
        
        # Get validation predictions
        y_prob = model.predict_proba(X_eval)[:, 1]
        metrics = optimize_threshold(model, y_eval, y_prob)
        
        return model, metrics
        
    except Exception as e:
        logger.error(f"Error training XGBoost model: {str(e)}")
        raise


In [187]:
# Block 7: Model Save/Load
def save_model(model, path, threshold=None):
    """Save XGBoost model and threshold to specified path."""
    path = Path(path)
    path.parent.mkdir(parents=True, exist_ok=True)
    
    try:
        # Save model
        joblib.dump(model, path)
        
        # Save threshold
        if threshold:
            threshold_path = path.parent / "threshold.json"
            with open(threshold_path, 'w') as f:
                json.dump({
                    'threshold': threshold,
                    'model_type': 'xgboost',
                    'params': model.get_params()
                }, f, indent=2)
                
        logger.info(f"Model saved to {path}")
        
    except Exception as e:
        logger.error(f"Error saving model: {str(e)}")
        raise

def load_model(path):
    """Load XGBoost model from specified path."""
    path = Path(path)
    if not path.exists():
        raise FileNotFoundError(f"No model file found at {path}")
        
    try:
        # Load model
        model = joblib.load(path)
        
        # Load threshold
        threshold_path = path.parent / "threshold.json"
        if threshold_path.exists():
            with open(threshold_path, 'r') as f:
                data = json.load(f)
                threshold = data.get('threshold', 0.5)
        else:
            threshold = 0.5
            
        logger.info(f"Model loaded from {path}")
        return model, threshold
        
    except Exception as e:
        logger.error(f"Error loading model: {str(e)}")
        raise


In [188]:
# Block 8: Hyperparameter Tuning
def objective(trial):
    """Optuna objective function for hyperparameter optimization."""
    try:
        params = {
            'objective': 'binary:logistic',
            # 'tree_method': 'hist',  # CPU-only as per requirements
            'eval_metric': ['logloss', 'auc'],
            'verbose': -1,
            'n_jobs': -1,
            'booster': 'dart',
            'normalize_type': 'tree',
            'sample_type': 'uniform',
            'random_state': 19
        }
        
        # Add hyperparameters from config
        hyperparameter_space = load_hyperparameter_space()
        for param_name, param_config in hyperparameter_space.items():
            if param_config['type'] == 'float':
                params[param_name] = trial.suggest_float(
                    param_name,
                    param_config['low'], 
                    param_config['high'],
                    log=param_config.get('log', False)  # Pass log if in param_config, default False
                )
            elif param_config['type'] == 'int':
                params[param_name] = trial.suggest_int(
                    param_name,
                    param_config['low'],
                    param_config['high']
                )
            elif param_config['type'] == 'categorical':
                params[param_name] = trial.suggest_categorical(
                    param_name,
                    param_config['choices']
                )
        # Train model and get metrics
        model, metrics = train_model(
            X_train, y_train,
            X_test, y_test,
            X_eval, y_eval,
            **params
        )
        
        recall = metrics.get('recall', 0.0)
        precision = metrics.get('precision', 0.0)
        
        # Report intermediate values for pruning
        trial.report(precision, step=1)
        if trial.should_prune():
            raise optuna.TrialPruned()
        
        # Optimize for precision while maintaining minimum recall
        score = precision if recall >= 0.15 else 0.0
        
        logger.info(f"Trial {trial.number}:")
        logger.info(f"  Params: {params}")
        logger.info(f"  Score: {score}")
        
        for metric_name, metric_value in metrics.items():
            trial.set_user_attr(metric_name, metric_value)
        return score

    except Exception as e:
        logger.error(f"Error in trial {trial.number}: {str(e)}")
        raise


In [189]:
# Block 9: Hypertuning Function
def hypertune_xgboost(experiment_name: str) -> float:
    """Run hyperparameter optimization with MLflow tracking."""
    try:
        # Create study
        study = optuna.create_study(
            study_name=f"xgboost_optimization_{datetime.now().strftime('%Y%m%d_%H%M')}",
            direction="maximize",
            sampler=TPESampler(seed=42),
            pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=5)
        )
        
        # Start MLflow run
        with mlflow.start_run(run_name=f"xgboost_optimization_{datetime.now().strftime('%Y%m%d_%H%M')}"):
            # Log dataset info
            mlflow.log_params({
                "train_samples": len(X_train),
                "test_samples": len(X_test),
                "eval_samples": len(X_eval),
                "features": X_train.shape[1]
            })
            
            # Set tags
            mlflow.set_tags({
                "model_type": "xgboost_base",
                "optimization": "optuna",
                "cpu_only": True
            })
            
            # Optimize
            study.optimize(objective, n_trials=50, timeout=7200)  # 2 hours timeout
            
            # Log best trial info
            logger.info(f"Best trial value: {study.best_value}")
            logger.info(f"Best parameters found: {study.best_params}")
            
            # Train final model with best parameters
            logger.info("Training final model with best parameters")
            final_model, final_metrics = train_model(
                X_train, y_train,
                X_test, y_test,
                X_eval, y_eval,
                **study.best_params
            )
            
            # Log best parameters and metrics
            mlflow.log_params(study.best_params)
            mlflow.log_metrics(final_metrics)
            
            # Create and log model signature
            input_example = pd.DataFrame(X_train[:1].copy())
            signature = mlflow.models.infer_signature(
                model_input=input_example,
                model_output=final_model.predict_proba(input_example)
            )
            
            # Log model
            mlflow.xgboost.log_model(
                xgb_model=final_model,
                artifact_path="xgboost_base_model",
                registered_model_name=f"xgboost_base_{datetime.now().strftime('%Y%m%d_%H%M')}",
                signature=signature,
                input_example=input_example
            )
            
            # Save study results
            study_path = Path(mlruns_dir) / experiment_name / "optuna_studies"
            study_path.mkdir(parents=True, exist_ok=True)
            joblib.dump(study, study_path / f"study_{datetime.now().strftime('%Y%m%d_%H%M')}.pkl")
            
            logger.info(f"Training completed with precision: {final_metrics['precision']:.4f}")
            return final_metrics['precision']
            
    except Exception as e:
        logger.error(f"Error in hyperparameter optimization: {str(e)}")
        raise


In [190]:
def train_with_precision_target(
    X_train: np.ndarray,
    y_train: np.ndarray,
    X_test: np.ndarray,
    y_test: np.ndarray,
    X_eval: np.ndarray,
    y_eval: np.ndarray,
    logger: ExperimentLogger) -> Tuple[Any, float, Dict[str, Any]]:
    """Train XGBoost model with target precision threshold."""
    
    precision = 0
    best_precision = 0
    best_recall = 0
    best_params = None
    best_seed = 0
    best_model = None
    best_threshold = 0.5
    
    # Base parameters from previous optimization
    base_params = {
        'objective': 'binary:logistic',
        'eval_metric': ['logloss', 'auc'],
        'booster': 'dart',
        'normalize_type': 'tree',
        'sample_type': 'uniform',
        'learning_rate': 0.030180820828066838,
        'max_depth': 14,
        'min_child_weight': 87,
        'subsample': 0.579597545259111,
        'colsample_bytree': 0.6312037280884872,
        'reg_alpha': 0.020511104188433976,
        'reg_lambda': 7.2226145454655,
        'gamma': 0.8795585311974417,
        'early_stopping_rounds': 330,
        'scale_pos_weight': 2.541614515559209,
        'rate_drop': 0.03555781345986666,
        'skip_drop': 0.7819459112971967,
        'tree_method': 'hist',
        'verbose': -1,
        'n_jobs': -1
    }
    
    while best_precision < 0.48:  # Target precision threshold
        for random_seed in range(1, 50):  # Try up to 1000 different seeds
            logger.info(f"Using sequential random seed: {random_seed}")
            
            # Set all random seeds
            os.environ['PYTHONHASHSEED'] = str(random_seed)
            np.random.seed(random_seed)
            random.seed(random_seed)
            base_params['random_state'] = random_seed
            
            try:
                # Create and train model
                model, metrics = train_model(
                    X_train, y_train,
                    X_test, y_test,
                    X_eval, y_eval,
                    **base_params
                )
                precision = metrics['precision']
                recall = metrics['recall']

                # Update best model if precision improved
                if precision > best_precision:
                    best_precision = precision
                    best_recall = recall
                    best_params = base_params.copy()
                    best_seed = random_seed
                    best_model = model
                    logger.info(f"New best precision: {precision:.4f} with seed {best_seed}")
                
                # Check if target precision reached
                if precision >= 0.48:
                    logger.info(f"Target precision achieved: {precision:.4f}")
                    return best_model, precision, recall, best_params
                
                logger.info(
                    f"Current precision: {precision:.4f}, "
                    f"target: 0.4800, highest precision: {best_precision:.4f}, "
                    f"best seed: {best_seed}"
                )
                
            except Exception as e:
                logger.error(f"Error training with seed {random_seed}: {str(e)}")
                continue
            
            # Clear model to free memory
            model = None
        
        # If target not reached after all seeds, return best model
        if precision < 0.48:
            logger.info(f"Target precision not reached, using best seed: {best_seed}")
            return best_model, best_precision, best_recall, best_params
            
    return best_model, best_precision, best_recall, best_params


In [191]:
def log_to_mlflow(model: object, precision: float, recall: float, params: dict, experiment_name: str) -> str:
    """Log model, metrics and parameters to MLflow.
    
    Args:
        model: Trained XGBoost model
        metrics (dict): Dictionary of metrics like precision, recall etc.
        params (dict): Model parameters used for training
        experiment_name (str): Name of the MLflow experiment
    """
    from utils.create_evaluation_set import setup_mlflow_tracking
    
    mlruns_dir = setup_mlflow_tracking(experiment_name)
    
    # Start MLflow run
    with mlflow.start_run(run_name=f"xgboost_base_{datetime.now().strftime('%Y%m%d_%H%M')}") as run:
        
        # Log parameters
        mlflow.log_params(params)
        
        # Log metrics
        mlflow.log_metrics({
            "precision": precision,
            "recall": recall
        })
        
        # Register model with timestamp
        model_name = f"xgboost_base_{datetime.now().strftime('%Y%m%d_%H%M')}"
        
        # Log model with signature
        input_example = pd.DataFrame(model.feature_names_in_[:1].copy(), dtype=float)
        signature = mlflow.models.infer_signature(
            model_input=input_example,
            model_output=predict_proba(model, input_example)
        )
        
        mlflow.xgboost.log_model(
            xgb_model=model,
            artifact_path="model",
            registered_model_name=model_name,
            signature=signature
        )
        
        # Log run ID
        run_id = run.info.run_id
        logger.info(f"Run ID: {run_id}")
        return run_id


In [192]:
def train_seed_model():
    model, precision, recall, best_params = train_with_precision_target(
                X_train, y_train,
                X_test, y_test,
                X_eval, y_eval,
                logger
            )
    print(f"Training completed with precision: {precision:.4f}")
    
    # Log to MLflow if we got a valid model
    if model is not None:
        log_to_mlflow(model, precision, recall, best_params, experiment_name)


In [193]:
if __name__ == "__main__":
    train_seed_model()
    # hypertune_xgboost(experiment_name)


[I 2025-02-20 21:34:50,803] A new study created in memory with name: xgboost_optimization_20250220_2134


2025-02-20 21:35:37,336 | INFO     | xgboost_soccer_prediction | Optimized threshold: 0.550 with precision: 0.389
2025-02-20 21:35:37,439 | INFO     | xgboost_soccer_prediction | Trial 0:
2025-02-20 21:35:37,441 | INFO     | xgboost_soccer_prediction |   Params: {'objective': 'binary:logistic', 'eval_metric': ['logloss', 'auc'], 'verbose': -1, 'n_jobs': -1, 'booster': 'dart', 'normalize_type': 'tree', 'sample_type': 'uniform', 'random_state': 19, 'learning_rate': 0.030180820828066838, 'max_depth': 14, 'min_child_weight': 87, 'subsample': 0.579597545259111, 'colsample_bytree': 0.6312037280884872, 'reg_alpha': 0.020511104188433976, 'reg_lambda': 7.2226145454655, 'gamma': 0.8795585311974417, 'early_stopping_rounds': 330, 'scale_pos_weight': 2.541614515559209, 'rate_drop': 0.03555781345986666, 'skip_drop': 0.7819459112971967}
2025-02-20 21:35:37,442 | INFO     | xgboost_soccer_prediction |   Score: 0.3887468030689543


[I 2025-02-20 21:35:37,446] Trial 0 finished with value: 0.3887468030689543 and parameters: {'learning_rate': 0.030180820828066838, 'max_depth': 14, 'min_child_weight': 87, 'subsample': 0.579597545259111, 'colsample_bytree': 0.6312037280884872, 'reg_alpha': 0.020511104188433976, 'reg_lambda': 7.2226145454655, 'gamma': 0.8795585311974417, 'early_stopping_rounds': 330, 'scale_pos_weight': 2.541614515559209, 'rate_drop': 0.03555781345986666, 'skip_drop': 0.7819459112971967}. Best is trial 0 with value: 0.3887468030689543.


2025-02-20 21:36:21,622 | INFO     | xgboost_soccer_prediction | Optimized threshold: 0.520 with precision: 0.360
2025-02-20 21:36:21,682 | INFO     | xgboost_soccer_prediction | Trial 1:
2025-02-20 21:36:21,685 | INFO     | xgboost_soccer_prediction |   Params: {'objective': 'binary:logistic', 'eval_metric': ['logloss', 'auc'], 'verbose': -1, 'n_jobs': -1, 'booster': 'dart', 'normalize_type': 'tree', 'sample_type': 'uniform', 'random_state': 19, 'learning_rate': 0.049912126454217776, 'max_depth': 9, 'min_child_weight': 59, 'subsample': 0.45502135295603013, 'colsample_bytree': 0.6608484485919075, 'reg_alpha': 0.11207606211860566, 'reg_lambda': 8.83505209874092, 'gamma': 0.36210622617823773, 'early_stopping_rounds': 331, 'scale_pos_weight': 2.4278987721304084, 'rate_drop': 0.1088790551045089, 'skip_drop': 0.4198171059762151}
2025-02-20 21:36:21,687 | INFO     | xgboost_soccer_prediction |   Score: 0.360381861575093


[I 2025-02-20 21:36:21,689] Trial 1 finished with value: 0.360381861575093 and parameters: {'learning_rate': 0.049912126454217776, 'max_depth': 9, 'min_child_weight': 59, 'subsample': 0.45502135295603013, 'colsample_bytree': 0.6608484485919075, 'reg_alpha': 0.11207606211860566, 'reg_lambda': 8.83505209874092, 'gamma': 0.36210622617823773, 'early_stopping_rounds': 331, 'scale_pos_weight': 2.4278987721304084, 'rate_drop': 0.1088790551045089, 'skip_drop': 0.4198171059762151}. Best is trial 0 with value: 0.3887468030689543.


...

2025-02-20 21:51:16,948 | INFO     | xgboost_soccer_prediction | Optimized threshold: 0.500 with precision: 0.365
2025-02-20 21:51:16,997 | INFO     | xgboost_soccer_prediction | Trial 21:
2025-02-20 21:51:16,999 | INFO     | xgboost_soccer_prediction |   Params: {'objective': 'binary:logistic', 'eval_metric': ['logloss', 'auc'], 'verbose': -1, 'n_jobs': -1, 'booster': 'dart', 'normalize_type': 'tree', 'sample_type': 'uniform', 'random_state': 19, 'learning_rate': 0.02903489413278031, 'max_depth': 13, 'min_child_weight': 81, 'subsample': 0.6478895373632062, 'colsample_bytree': 0.6162487607802904, 'reg_alpha': 0.3756615413011831, 'reg_lambda': 7.365523684174454, 'gamma': 0.7906242212416662, 'early_stopping_rounds': 327, 'scale_pos_weight': 2.5342779918997067, 'rate_drop': 0.15173900988169994, 'skip_drop': 0.7310256004535236}
2025-02-20 21:51:17,000 | INFO     | xgboost_soccer_prediction |   Score: 0.3645418326692501


[I 2025-02-20 21:51:17,003] Trial 21 finished with value: 0.3645418326692501 and parameters: {'learning_rate': 0.02903489413278031, 'max_depth': 13, 'min_child_weight': 81, 'subsample': 0.6478895373632062, 'colsample_bytree': 0.6162487607802904, 'reg_alpha': 0.3756615413011831, 'reg_lambda': 7.365523684174454, 'gamma': 0.7906242212416662, 'early_stopping_rounds': 327, 'scale_pos_weight': 2.5342779918997067, 'rate_drop': 0.15173900988169994, 'skip_drop': 0.7310256004535236}. Best is trial 0 with value: 0.3887468030689543.


2025-02-20 21:52:00,807 | INFO     | xgboost_soccer_prediction | Optimized threshold: 0.490 with precision: 0.329
2025-02-20 21:52:00,850 | INFO     | xgboost_soccer_prediction | Trial 22:
2025-02-20 21:52:00,852 | INFO     | xgboost_soccer_prediction |   Params: {'objective': 'binary:logistic', 'eval_metric': ['logloss', 'auc'], 'verbose': -1, 'n_jobs': -1, 'booster': 'dart', 'normalize_type': 'tree', 'sample_type': 'uniform', 'random_state': 19, 'learning_rate': 0.02431725951489529, 'max_depth': 14, 'min_child_weight': 70, 'subsample': 0.6041698717741358, 'colsample_bytree': 0.6362967551069821, 'reg_alpha': 0.24686319407541057, 'reg_lambda': 7.056696953907927, 'gamma': 0.462438855590888, 'early_stopping_rounds': 334, 'scale_pos_weight': 2.575921309596674, 'rate_drop': 0.2145363618751568, 'skip_drop': 0.5776290911335666}
2025-02-20 21:52:00,853 | INFO     | xgboost_soccer_prediction |   Score: 0.3291316526610183


[I 2025-02-20 21:52:00,855] Trial 22 finished with value: 0.3291316526610183 and parameters: {'learning_rate': 0.02431725951489529, 'max_depth': 14, 'min_child_weight': 70, 'subsample': 0.6041698717741358, 'colsample_bytree': 0.6362967551069821, 'reg_alpha': 0.24686319407541057, 'reg_lambda': 7.056696953907927, 'gamma': 0.462438855590888, 'early_stopping_rounds': 334, 'scale_pos_weight': 2.575921309596674, 'rate_drop': 0.2145363618751568, 'skip_drop': 0.5776290911335666}. Best is trial 0 with value: 0.3887468030689543.
[W 2025-02-20 21:52:11,198] Trial 23 failed with parameters: {'learning_rate': 0.03902088786039971, 'max_depth': 13, 'min_child_weight': 90, 'subsample': 0.6689778147765486, 'colsample_bytree': 0.6002847944241311, 'reg_alpha': 0.562336473344184, 'reg_lambda': 7.791242309411482, 'gamma': 0.8484086311810264, 'early_stopping_rounds': 325, 'scale_pos_weight': 2.466646339329373, 'rate_drop': 0.16405715958915698, 'skip_drop': 0.7854840624503119} because of the following error:

KeyboardInterrupt: 